In [1]:
import math
import sympy as sym
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from sympy.abc import beta
from scipy.integrate import quad
from scipy.special import gamma
from sympy import I

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>#notebook-container { padding-bottom: 901px ; }</style>"))

In [2]:
from sympy import legendre, diff
th, R = sym.symbols('th, r')

# Axial multipole expansion

$V = K' \sum_{l=0}^{\inf} \frac{\Sigma_i q_i r_i^l P_l (\cos(\theta_i )}{R^{l+1}}$

This sum does not converge for $r/R > 1$. A different element can be factored out befor the Legendre polynomial substitution for this case.

In [84]:
def axial_ex(point_charges, p0):
    ep0 = 8.8541878128E-12
    K = 1/(4*np.pi*ep0)
    E = [0,0]
    for j in point_charges:
        V = 0
        for i in range(50):
            V += (np.linalg.norm(j[0:3])**i/R**(i+1)) * legendre(i, sym.cos(th))
        ER =  - diff(V, R)
        Eth =  - (1/R) * diff(V, th)
        lam_ER = sym.lambdify([R, th], ER)
        lam_Eth = sym.lambdify([R, th], Eth)
        theta = np.arccos(np.dot(j[0:3], p0)/(np.linalg.norm(j[0:3])*np.linalg.norm(p0)))
        m = p0[1]/p0[0]
        if m*j[0] < j[1]:  ### Checks if angle is greater than pi/2
            theta = -1*theta
        E[0] += K * j[-1] * lam_ER(np.linalg.norm(p0), theta)
        E[1] += K * j[-1] * lam_Eth(np.linalg.norm(p0), theta)
    return E ### [E_R, E_theta]

In [51]:
point_charges = np.array([[1,0,0, 4E-9]]) ### [X,Y,Z, Q]
p0 = np.array([10, 2, 0]) ### Point of interest
E = axial_ex(point_charges, p0)
### Transform into Cartesian
theta = np.arctan(p0[1]/p0[0])
E[0]*np.cos(theta)+E[1]*np.cos(theta +np.pi/2)
E[0]*np.sin(theta)+E[1]*np.sin(theta +np.pi/2)

0.19739555984988044
0.19739555984988044


0.4128720829335742

0.09174935176301649

In [37]:
def E_field(r, rp, q):
    k = 8.98755262e9
    
    R = []
    for i in range(len(r)):
        R.append(rp[i] - r[i])
        
    rmag = r_mag(R)
    
    E = np.array([0.0,0.0,0.0])
    for i in range(E.size):
        E[i] = (k*q*rmag**-3*R[i])
    
    return E
        
    
def r_mag(r):
    rmag = 0
    for i in r:
        rmag += i**2
    return rmag**.5

In [38]:
r = [1, 0, 0]
rp = [10, 2, 0]
q = 4E-9
E_field(r, rp, q)

array([0.41287212, 0.09174936, 0.        ])

In [85]:
point_charges = np.array([[1,0,0, 4E-8],  
                          [2,0,0, -3E-8], ### list of point charges
                          [-5,2,0, 5E-8]])

p0 = np.array([10, 0, 0]) ### Point of interest
E = axial_ex(point_charges, p0)
E

theta = np.arctan(p0[1]/p0[0])
theta
E[0]*np.cos(theta)+E[1]*np.cos(theta +np.pi/2)
E[0]*np.sin(theta)+E[1]*np.sin(theta +np.pi/2)

E_test = np.zeros(3)
for i in range(point_charges.shape[0]):
    E_test += E_field(point_charges[i][0:3], p0, point_charges[i][-1])
E_test

[2.170515923997889, -0.2593511527004362]

0.0

2.170515923997889

-0.2593511527004362

array([ 2.17051612, -0.25935118,  0.        ])

In [11]:
E = np.array([0.0,0.0,0.0])
E[:] = 0.5
E

array([0.5, 0.5, 0.5])